In [1]:
pic_='F'
from itertools import repeat
import pickle5 as pickle
import pandas as pd
from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_points_griddata, rasterize_points_radial
import rioxarray as rio
import rasterio
import numpy as np
import geopandas as gpd
from multiprocessing import Pool
import os
from rasterio.merge import merge
import itertools

In [2]:
##Load data and model
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +".csv",sep=',')
grid_20=data_sample.groupby(['ID_GRID'])['ID_GRID'].count().index.to_list()[:30]

with open('model/model-result/model_final.pickle', 'rb') as handle:
    best_=pickle.load(handle)
best_model=best_[0]
best_fs=best_[1]

In [3]:
best_model

StackingClassifier(estimators=[('catboost',
                                <catboost.core.CatBoostClassifier object at 0x7fe3c7f3df40>),
                               ('lightgbm',
                                LGBMClassifier(bagging_fraction=0.4,
                                               bagging_freq=1,
                                               feature_fraction=0.4,
                                               lambda_l1=0.031499974612702875,
                                               lambda_l2=0.031499974612702875,
                                               learning_rate=0.07717099117951648,
                                               max_depth=4,
                                               min_data_in_leaf=1200,
                                               min_gain_to_split=0.6794386913108047,
                                               n_estimators=500,
                                               num_leaves=2820,
                                               objective='MultiClass',
                                               random_seed=1234)),
                               ('randomforest',
                                RandomForestClassifier(criterion='entropy',
                                                       max_depth=8,
                                                       max_leaf_nodes=12,
                                                       min_samples_leaf=32,
                                                       min_samples_split=96,
                                                       random_state=1234))],
                   final_estimator=LogisticRegression())

In [4]:
best_fs

['B1_p15',
 'B2_p15',
 'B3_p15',
 'B4_p15',
 'B5_p15',
 'B7_p15',
 'B8A_p15',
 'B11_p15',
 'B12_p15',
 'NDVI_p50',
 'NDWI_p50',
 'NDBI_p50',
 'SAVI_p50',
 'EVI_p50',
 'GNDVI_p50',
 'B6_p15',
 'B8_p15']

In [5]:
def parallel_classify_raster(idgrid):
    global best_model,best_fs,pic_
    #print(i)
    #print(j)
    name_image='duatahun_'+str(idgrid)+'_QALPN1_PakKus_sentinel2_10m.tif'
    data_riox=rio.open_rasterio('gs://bps-gcp-bucket/citra-sentinel2/'+pic_+'/'+name_image)
    min_x=min(data_riox.x).values+0
    max_x=max(data_riox.x).values+0
    min_y=min(data_riox.y).values+0
    max_y=max(data_riox.y).values+0
    x_=list(np.linspace(min_x,max_x,10))
    y_=list(np.linspace(min_y,max_y,10))
    rangex=range(0,len(x_)-1)
    rangey=range(0,len(y_)-1)
    for i in rangex:
        for j in rangey:
            t_data_riox=data_riox.rio.clip_box(minx=x_[i],miny=y_[j],maxx=x_[i+1],maxy=y_[j+1])
            temp_=t_data_riox.to_dataframe(name='value').reset_index().fillna(0)
            matrixData=temp_[['x','y']].drop_duplicates()
            for h in temp_.band.unique():
                t_=temp_.loc[temp_.band==h,['x','y','value']].rename(columns={'value':data_riox.long_name[h-1]})
                matrixData=matrixData.merge(t_,how='left')
            matrixData=gpd.GeoDataFrame(matrixData, crs='epsg:4326',
                                        geometry=gpd.points_from_xy(matrixData.x,matrixData.y)).to_crs('epsg:3857')
            matrixData['prediction']=best_model.predict(matrixData[best_fs])
            matrixData['prediction']=matrixData['prediction'].astype(np.int32)
            geo_grid = make_geocube(
                            vector_data=matrixData,
                            measurements=['prediction'],
                            resolution=(-10, 10),
                            rasterize_function=rasterize_points_griddata)
            geo_grid.prediction.rio.write_nodata(0,inplace=True)
            tif_name='classified_'+str(idgrid)+'_'+str(i)+'-'+str(j)+'.tif'
            geo_grid.rio.to_raster('class_result/temp/'+tif_name)
    ls_file=os.listdir('class_result/temp')
    src_files_to_mosaic = []
    tif_name='class_result/result/classified_'+str(idgrid)+'.tif'
    ls_y=[y for y in ls_file if y.replace('classified_','').split('_')[0]==str(idgrid)]
    for ls in ls_y:
        src = rasterio.open('class_result/temp/'+ls)
        src_files_to_mosaic.append(src)
    mosaic, out_trans = merge(src_files_to_mosaic)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff","height": mosaic.shape[1],"width": mosaic.shape[2],"transform": out_trans})
    with rasterio.open(tif_name, "w", **out_meta) as dest:
        dest.write(mosaic)
        print(tif_name, 'Done')

In [6]:
grid_20

['ID-3043',
 'ID-3044',
 'ID-3086',
 'ID-3087',
 'ID-3128',
 'ID-3129',
 'ID-3130',
 'ID-3173',
 'ID-3174',
 'ID-3175',
 'ID-3177',
 'ID-3222',
 'ID-3223',
 'ID-3224',
 'ID-3225',
 'ID-3270',
 'ID-3271',
 'ID-3272',
 'ID-3273',
 'ID-3324',
 'ID-3325',
 'ID-3326',
 'ID-3443',
 'ID-3496',
 'ID-3497',
 'ID-3498',
 'ID-3499',
 'ID-3549',
 'ID-3550',
 'ID-3551']

In [ ]:
for i in grid_20:
    parallel_classify_raster(i)

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: Ras

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: Ras

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: Ras

class_result/result/classified_ID-3043.tif Done


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: Ras

...

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: Ras

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: Ras

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: Ras

class_result/result/classified_ID-3175.tif Done


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/rasterio/windows.py:310: Ras